In [45]:
import pandas as pd
import numpy as np

def gen_serve_error(player, path):
    # Load the data
    events = pd.read_csv(path)
    events['pointWonBy'] = events.groupby('pointNumber')['pointWonBy'].bfill()

    # Filter for returns by the specified player
    returns = events[(events['serverName'] == player) & (events['shotInRally'] == 1)].copy()

    # Flip y and x coordinates where y > 0
    returns['firstServeYCoord'] = returns['firstServeYCoord'].apply(lambda y: -y if y > 0 else y)
    returns['secondServeYCoord'] = returns['secondServeYCoord'].apply(lambda y: -y if y > 0 else y)
    
    returns['firstServeXCoord'] = returns.apply(lambda row: -row['firstServeXCoord'] if row['firstServeYCoord'] < 0 else row['firstServeXCoord'], axis=1)
    returns['secondServeXCoord'] = returns.apply(lambda row: -row['secondServeXCoord'] if row['secondServeYCoord'] < 0 else row['secondServeXCoord'], axis=1)

    # Round coords near net
    returns['firstServeYCoord'] = returns['firstServeYCoord'].apply(lambda y: 0 if y >= -25 else y)
    returns['secondServeYCoord'] = returns['secondServeYCoord'].apply(lambda y: 0 if y >= -25 else y)

    # Apply additional filtering for firstServeIn and firstServe/secondServeIn conditions
    returns = returns[
        (returns['firstServeIn'] != 1.0) | 
        ((returns['firstServeIn'] != 1.0) & (returns['secondServeIn'] != 1.0))
    ]

    # Create new columns 'x' and 'y' based on serve conditions
    returns['x'] = np.where(returns['firstServeIn'] != 1.0, returns['firstServeXCoord'], returns['secondServeXCoord'])
    returns['y'] = np.where(returns['firstServeIn'] != 1.0, returns['firstServeYCoord'], returns['secondServeYCoord'])

    # Duplicate rows where both firstServeIn and secondServeIn are not equal to 1.0
    double_errors = returns[(returns['firstServeIn'] != 1.0) & (returns['secondServeIn'] != 1.0)]
    if not double_errors.empty:
        first_serve_errors = double_errors.copy()
        second_serve_errors = double_errors.copy()

        # For first serve errors, use firstServe coordinates
        first_serve_errors['x'] = first_serve_errors['firstServeXCoord']
        first_serve_errors['y'] = first_serve_errors['firstServeYCoord']

        # For second serve errors, use secondServe coordinates
        second_serve_errors['x'] = second_serve_errors['secondServeXCoord']
        second_serve_errors['y'] = second_serve_errors['secondServeYCoord']

        # Combine original, first serve errors, and second serve errors
        returns = pd.concat([returns, first_serve_errors, second_serve_errors], ignore_index=True)
    else:
        returns = pd.concat([returns, double_errors], ignore_index=True)

    # Select only the specified columns
    serve_errors = returns[['serverName', 'firstServeIn', 'secondServeIn', 
                             'x', 'y']]
    # Export the data
    serve_errors_json = serve_errors.to_json(orient='records')

    with open('serve_errors.json', 'w') as f:
        f.write(serve_errors_json)

    print(serve_errors)

In [46]:
player = "Elise Wagle"
path = '../Shot_Visuals_EliseWagle_MotkaKomac.csv'
gen_serve_error(player, path)

     serverName  firstServeIn  secondServeIn      x      y
0   Elise Wagle           0.0            1.0  -19.0 -381.0
1   Elise Wagle           0.0            1.0   56.0 -323.0
2   Elise Wagle           0.0            0.0  168.0 -226.0
3   Elise Wagle           0.0            0.0  -26.0 -260.0
4   Elise Wagle           0.0            0.0   99.0 -287.0
5   Elise Wagle           0.0            1.0  -88.0    0.0
6   Elise Wagle           0.0            1.0  -37.0 -212.0
7   Elise Wagle           0.0            1.0 -103.0    0.0
8   Elise Wagle           0.0            1.0   10.0    0.0
9   Elise Wagle           0.0            0.0  -74.0 -249.0
10  Elise Wagle           0.0            0.0   91.0 -254.0
11  Elise Wagle           0.0            1.0   17.0    0.0
12  Elise Wagle           0.0            1.0  -73.0    0.0
13  Elise Wagle           0.0            1.0   36.0    0.0
14  Elise Wagle           0.0            1.0  -94.0    0.0
15  Elise Wagle           0.0            1.0  177.0 -369